# Handling multi-dimensional arrays with xarray

## Authors & Contributors

### Authors

- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)
- Anne Fouilloux, Simula Research Laboratory (Norway), [@annefou](https://github.com/annefou)

### Contributors

- Alejandro Coca-Castro, The Alan Turing Institute (United Kingdom), [@acocac](https://github.com/acocac)
- Guillaume Eynard-Bontemps, CNES (France), [@guillaumeeb](https://github.com/guillaumeeb)

<div class="alert alert-info">
    <i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> <b>Overview</b>
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>What is Xarray?</li>
        <li>How to open a local file?</li>
        <li>How to print metadata information?</li>
        <li>How to make a selection?</li>
        <li>How to visualize with matplotlib?</li>
        <li>How to perform basic computations, statistics and aggregations?</li>
        <li>How to mask data?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about Xarray Python ecosystem</li>
        <li>Learn file handling with xarray</li>
        <li>Learn to get metadata information</li>
        <li>Learn to select and mask data</li>
        <li>Learn to make basic computations, aggregations and statistics</li>
        <li>Learnn to visualize data</li>
    </ul>
</div>

## Context

We will be using the [Pangeo](https://pangeo.io/) open-source software stack for visualizing the near-surface temperature and computing time averaged values (such as seasonal mean and other statistics).

### Data

In this episode, we will use [CMIP6](https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6) data.

This dataset can be discovered through the [CMIP6 online catalog](https://pangeo-data.github.io/pangeo-cmip6-cloud/) or from [ESGF](https://esgf.llnl.gov/).

The same dataset can also be downloaded from [Zenodo](https://zenodo.org/): [Near-surface Temperature from CMIP6 NCAR CESM2 historical monthly dataset for CLIVAR CMIP6 Bootcamp](https://zenodo.org/record/7181714/files/CMIP6_NCAR_CESM2_historical_amon_gn.nc).

## Setup

This episode uses the following main Python packages:

- xarray {cite:ps}`a-xarray-hoyer2017` with [`netCDF4`](https://pypi.org/project/h5netcdf/) and [`h5netcdf`](https://pypi.org/project/h5netcdf/) engines
- pooch {cite:ps}`a-pooch-Uieda2020`
- numpy {cite:ps}`a-numpy-harris2020`

Please install these packages if they are not already available in your Python environment (see [Setup page](https://pangeo-data.github.io/foss4g-2022/before/setup.html)).

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend that you install and import all the necessary libraries at the top of your Jupyter notebook.

In [ ]:
import xarray as xr

### Fetch Data

- For now we will fetch a netCDF file containing the near-surface temperature from one single CMIP6 model CESM2.
- The file is available in a Zenodo repository. We will download it using using `pooch`, a very handy Python-based library to download and cache your data files locally (see further info [here](https://www.fatiando.org/pooch/latest/index.html))
- In the [Data access and discovery](https://pangeo-data.github.io/clivar-2022/pangeo101/data_discovery.html) episode, we will learn about different ways to access data, including access to remote data.

In [ ]:
import pooch

In [ ]:
tas_file = pooch.retrieve(
    url="https://zenodo.org/record/7181714/files/CMIP6_NCAR_CESM2_historical_amon_gn.nc",
    known_hash="md5:5f86251e5bc5ef9b86a3a86cd06a536b",
    path=f".",
)

## Open and read metadata through Xarray

In [ ]:
tas_ds = xr.open_dataset(tas_file)

As the dataset is in the NetCDF format, Xarray automatically selects the correct engine (this happens in the background because engine='netcdf' has been automatically specified). Other common options are "h5netcdf" or "zarr".
GeoTiff data can also be read, but to access it requires rioxarray, which will be quickly covered later.
Supposing that you have a dataset in an unrecognised format, you can always create your own reader as a subclass of the backend entry point and pass it through the engine parameter.

:::{tip}
If you get an error with the previous command, first check the location of the input file some_hash-CMIP6_NCAR_CESM2_historical_amon_gn.nc: it should have been downloaded in the same directory as your Jupyter Notebook.
:::

In [ ]:
tas_ds

## What is xarray?

Xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like multi-dimensional arrays, which allows for a more intuitive, more concise, and less error-prone developer experience.

### How is xarray structured?

Xarray has two core data structures, which build upon and extend the core strengths of NumPy and Pandas libraries. Both data structures are fundamentally N-dimensional:

- [DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray) is the implementation of a labeled, N-dimensional array. It is an N-D generalization of a Pandas.Series. The name DataArray itself is borrowed from [Fernando Perez’s datarray project](http://fperez.org/py4science/datarray/), which prototyped a similar data structure.

- [Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) is a multi-dimensional, in-memory array database. It is a dict-like container of DataArray objects aligned along any number of shared dimensions, and serves a similar purpose in xarray as the pandas.DataFrame.


## Accessing Coordinates and Data Variables 

DataArray, within Datasets, can be accessed through:
- the dot notation like Dataset.NameofVariable  
- or using square brackets, like Dataset['NameofVariable'] (NameofVariable needs to be a string so use quotes or double quotes)

In [ ]:
tas_ds.time

`tas_ds.time` is a one-dimensional `xarray.DataArray` with dates of type `cftime.DatetimeNoLeap`

In [ ]:
tas_ds.tas

`tas_ds.tas` is a 4-dimensional `xarray.DataArray` with tas values of type `float32`

In [ ]:
tas_ds['tas']

Same can be achieved for attributes and a DataArray.attrs will return a dictionary.

In [ ]:
tas_ds['tas'].attrs

### Xarray and Memory usage

Once a Data Array|Set is opened, xarray loads into memory only the coordinates and all the metadata needed to describe it.
The underlying data, the component written into the datastore, are loaded into memory as a NumPy array, only once directly accessed; once in there, it will be kept to avoid re-readings.
This brings the fact that it is good practice to have a look to the size of the data before accessing it. A classical mistake is to try loading arrays bigger than the memory with the obvious result of killing a notebook Kernel or Python process.
If the dataset does not fit in the available memory, then the only option will be to load it through the chunking; later on, in the tutorial 'chunking_introduction', we will introduce this concept.

As the size of the data is not too big here, we can continue without any problem. But let's first have a look to the actual size and then how it impacts the memory once loaded into it.

In [ ]:
import numpy as np

In [ ]:
print(f'{np.round(tas_ds.tas.nbytes / 1024**2, 2)} MB') # all the data are automatically loaded into memory as NumpyArray once they are accessed.

In [ ]:
tas_ds.tas.data

## Renaming Coordinates and Data Variables 

It may be useful to rename variables or coordinates to more common ones. It is usually not necessary for CMIP6 data because coordinate and variable names are fully standardized. CMIP6 data follows [CF-conventions](https://cfconventions.org/). 

We will however show you how you can rename coordinates and/or variables and revert back our change.

In [ ]:
tas_ds = tas_ds.rename(lon='longitude', lat='latitude')

In [ ]:
tas_ds

Now let's revert back our change and rename latitude and longitude to the most common names e.g. 'lat' and 'lon'.

In [ ]:
tas_ds = tas_ds.rename(longitude='lon', latitude='lat')

In [ ]:
tas_ds

## Selection methods

As underneath DataArrays are Numpy Array objects (that implement the standard Python x[obj] (x: array, obj: int,slice) syntax). Their data can be accessed through the same approach of numpy indexing.

In [ ]:
tas_ds.tas[0,0,100,100]

or slicing

In [ ]:
tas_ds.tas[0, 0:5, 100:110, 100:110]

As it is not easy to remember the order of dimensions, Xarray really helps by making it possible to select the position using names:

- `.isel` -> selection based on positional index
- `.sel`  -> selection based on coordinate values

We first check the number of elements in each coordinate of the `tas` Data Variable using the built-in method sizes. Same result can be achieved querying each coordinate using the Python built-in function `len`.

In [ ]:
tas_ds.tas.sizes

In [ ]:
tas_ds.tas.isel(member_id=0,time=0, lat=100, lon=100) # same as tas_ds.tas[0,0,100,100]

The more common way to select a point is through the labeled coordinate using the `.sel` method.

In [ ]:
tas_ds.tas.sel(time='2010-01-15')

Time is easy to be used as there is a 1 to 1 correspondence with values in the index, float values are not that easy to be used and a small discrepancy can make a big difference in terms of results.


Coordinates are always affected by precision issues; the best option to quickly get a point over the coordinates is to set the sampling method (method='') that will search for the closest point according to the specified one.

Options for the method are:
- pad / **f**fill: propagate last valid index value forward
- backfill / **b**fill: propagate next valid index value backward
- nearest: use nearest valid index value

Another important parameter that can be set is the tolerance that specifies the distance between the requested and the target (so that abs(index\[indexer] - target) <= tolerance) from [documentation](https://xarray.pydata.org/en/v0.17.0/generated/xarray.DataArray.sel.html#:~:text=xarray.DataArray.sel%20%C2%B6%20DataArray.sel%28indexers%3DNone%2C%20method%3DNone%2C%20tolerance%3DNone%2C%20drop%3DFalse%2C%20%2A%2Aindexers_kwargs%29%20%C2%B6,this%20method%20should%20use%20labels%20instead%20of%20integers.).

In [ ]:
tas_ds.sel(lat=46.3, lon=8.8, method='nearest')

:::{warning}
To select a single real value without specifying a method, you would need to specify the exact encoded value; not the one you see when printed.
:::

In [ ]:
tas_ds.isel(lon=100).lon.values.item()

In [ ]:
tas_ds.isel(lat=100).lat.values.item()

In [ ]:
tas_ds.sel(lat=4.240837696335078, lon=125.0)

That is why we use a `method`! It makes your life easier to deal with inexact matches.

As the exercise is focused on an Area Of Interest, this can be obtained through a bounding box defined with slices.

In [ ]:
tas_AOI = tas_ds.tas.sel(lat=slice(54.5,71.5), lon=slice(-2.5,42.5))
tas_AOI

:::{tip} the values for `lat` and `lon` need to be selected in the order shown in the coordinate section (here in increasing order) and not always in increasing order?
**You need to use the same order as the corresponding DataArray**.
:::

## Plotting
   Plotting data can easily be obtained through matplotlib.pyplot back-end [matplotlib documentation](https://matplotlib.org/stable/index.html).

In [ ]:
tas_AOI.isel(time=0).plot(cmap="coolwarm")

In the next episode, we will learn more about advanced visualization tools and how to make interactive plots using [holoviews](https://holoviews.org/), a tool part of the [HoloViz](https://holoviz.org/) ecosystem.

## Basic maths

Near-surface temperature values are in Kelvin and we can easily convert them in degrees celcius.

Simple arithmetic operations can be performed without worrying about dimensions and coordinates, using the same notation we use with `numpy`. Underneath xarray will automatically vectorize the operations over all the data dimensions.

In [ ]:
tas_AOI - 273.15

The universal function ([ufunc](https://numpy.org/doc/stable/reference/ufuncs.html)) from numpy and scipy can be applied too directly to the data. There are currently more than 60 universal functions defined in numpy on one or more types, covering a wide variety of operations including math operations, trigonometric functions, etc.

In [ ]:
np.subtract(tas_AOI, 273.15)

In [ ]:
tas_AOI = tas_AOI - 273.15

In [ ]:
tas_AOI

## Statistics

All the standard statistical operations can be used such as `min`, `max`, `mean`. When no argument is passed to the function, the operation is done over all the dimensions of the variable (same as with `numpy`).

In [ ]:
tas_AOI.min()

You can make a statistical operation over a dimension. For instance, let's retrieve the maximum tas value among all those available for different times, at each lat-lon location.

In [ ]:
tas_AOI.max(dim='time')

## Aggregation
We have monthly data. To obtain yearly values, we can group values per year and compute the mean.

In [ ]:
tas_yearly = tas_AOI.groupby(tas_AOI.time.dt.year).mean()

In [ ]:
tas_yearly

As we have data from 1850 to 2014, the time dimension is now `year` and takes values from `1850` to `2014`.

In [ ]:
tas_yearly.year

## Mask

Masking can be achieved through the method `DataSet|Array.where(cond, other)` or `xr.where(cond, x, y)`.

The difference consists in the possibility to specify the value in case the condition is positive or not; `DataSet|Array.where(cond, other)` only offer the possibility to define the false condition value (by default is set to np.NaN))

In [ ]:
tas_masked = tas_AOI.where((tas_AOI >= -5.5) & (tas_AOI <= 15))

In [ ]:
tas_masked.isel(time=0).plot()

To better visualize the mask, with the help of `xr.where`, ad-hoc variables can be created. 'xr.where' lets us specify values of 1 for masked and 0 for the unmasked data.

In [ ]:
mask = xr.where((tas_AOI <= -5.5) | (tas_AOI >= 15), 1, 0)

In [ ]:
mask.isel(time=0).plot()

Plot a single point (defined by its latitude and longitude) over the time dimension.

In [ ]:
tas_masked.sel(lat=60., lon=10.75, method='nearest').plot()

## Save xarray Dataset

It is very often convenient to save intermediate or final results into a local file. We will learn more about the different file formats Xarray can handle, but for now let's save it as a netCDF file. Check the file size after saving the result into netCDF.

In [ ]:
tas_masked.to_netcdf('tas_Nordic_masked.nc')

## Advance Saving methods
### Encoding and Compression

From the near-surface temperature dataset we already know that values are encoded as `float32`. A compression method can be defined as well; if the format is netCDF4 with the engine set to 'netcdf4' or 'h5netcdf' there are different compression options.  The easiest solution is to stick with the default one for NetCDF4 files.

Note that encoding parameters needs to be done through a nested dictionary and parameters has to be defined for each single variable.

In [ ]:
tas_masked.to_netcdf('tas_Nordic_mcs.nc',
                      engine='netcdf4',
                      encoding={'tas':{"dtype": np.float32,
                                        'zlib': True, 'complevel':4}
                                }
                      )

<div class=\"alert alert-success\">
<i class=\"fa-check-circle fa\" style=\"font-size: 22px;color:#666;\"></i> <b>Key Points</b>
<br>
<ul>
   <li>Xarray Dataset and DataArray</li>
   <li>Read and get metadata from local raster file</li>
   <li>Dataset and DataArray selection</li>
   <li>Aggregation and statistics</li>
   <li>Masking values</li>
   </ul>
</div>

Through the datatype and the compression a compression of almost 10 time has been achieved; as drawback speed reading has been decreased.

## References

```{bibliography}
:style: alpha
:filter: topic % "xarray" and not topic % "package"
:keyprefix: a-
```

## Packages citation

```{bibliography}
:style: alpha
:filter: topic % "xarray" and topic % "package"
:keyprefix: a-
```